In [104]:
class Block:
    
    def __init__(self,name):
        self.name = name
    
    def render(self, i):
        i += 1
        return "X{}[{}]".format(i, self.name), i
        
class ParallelBlock:
    def __init__(self, name, components):
        self.name = name
        self.components = components
    
    def render(self, i):
        start_i = i
        out = ""
        
        nodes = []
        
        # Render components
        for component in self.components:
            component, i = component.render(i)
            out += component
            out += "\n"
            nodes.append(i)
        end_i = i
        
        # Render invisible connections
        for ix1, ix2 in zip(nodes[:-1], nodes[1:]):
            out += "X{} ~~~ X{}".format(ix1, ix2)
            out += "\n"
        
        
        # Subgraph it
        i += 1
        out = "subgraph X{}[{}]\n".format(i, self.name) + out
        out += "end"
        
        return out, i
        
class StackBlock:
    def __init__(self, name, components):
        self.name = name
        self.components = components
        
    def render(self, i):
        start_i = i
        out = ""
        
        nodes = []
        
        # Render components
        for component in self.components:
            component, i = component.render(i)
            out += component
            out += "\n"
            nodes.append(i)
        end_i = i
        
        # Render connections
        for ix1, ix2 in zip(nodes[:-1], nodes[1:]):
            out += "X{}-->X{}".format(ix1, ix2)
            out += "\n"
        
        # Subgraph it
        i += 1
        out = "subgraph X{}[{}]\n".format(i, self.name) + out
        out += "end"
        
        return out, i

In [105]:
block_a = Block(name='A')
block_b = Block(name='B')
block_c = Block(name='C')
block_d = Block(name='D')
block_e = Block(name='E')

In [106]:
simulation1_block = StackBlock('Simulation 1', [block_a, block_b, block_c])
i = 0
g, i = simulation1_block.render(i)
print(g)

subgraph X4[Simulation 1]
X1[A]
X2[B]
X3[C]
X1-->X2
X2-->X3
end


In [107]:
simulation2_block = ParallelBlock('Simulation 2', [block_a, block_b, block_c])
i = 0
g, i = simulation2_block.render(i)
print(g)

subgraph X4[Simulation 2]
X1[A]
X2[B]
X3[C]
X1 ~~~ X2
X2 ~~~ X3
end


In [103]:
parallel_block = ParallelBlock('Parallel Block', [block_a, block_b])
stack_block = StackBlock('Stack Block', [block_c, block_d])
simulation3_block = StackBlock("Simulation 3", [parallel_block, stack_block, block_e])
i = 0
g, i = simulation3_block.render(i)
print(g)

subgraph X8[Simulation 3]
subgraph X3[Parallel Block]
X1[A]
X2[B]
X1 ~~~ X2
end
subgraph X6[Stack Block]
X4[C]
X5[D]
X4-->X5
end
X7[E]
X1-->X2
X2-->X3
X3-->X4
X4-->X5
X5-->X6
X6-->X7
end


In [ ]:
simulation2_block = StackBlock('Simulation 2', [block_a, block_b, block_c])
i = 0
g, i = simulation1_block.render(i)
print(g)

```mermaid
graph LR
A[JSON Object \n\n Each spec has a repo for tracking changes \n Must conform to the json specification \n Defines all aspects of the spec including blocks, spaces and actions] -->B[MSML Object \n\n JSON file and parsed, with validations and mappings along the way \n Can show different views on the fly]
    B --> C[Report Outputs \n\n Automatically build reports for the full spec or subviews \n Example: all blocks with an effect on variable XYZ]

```

In [19]:
from IPython.display import display

display("""```mermaid
graph LR
A[JSON Object \n\n Each spec has a repo for tracking changes \n Must conform to the json specification \n Defines all aspects of the spec including blocks, spaces and actions] -->B[MSML Object \n\n JSON file and parsed, with validations and mappings along the way \n Can show different views on the fly]
    B --> C[Report Outputs \n\n Automatically build reports for the full spec or subviews \n Example: all blocks with an effect on variable XYZ]

```""")

'```mermaid\ngraph LR\nA[JSON Object \n\n Each spec has a repo for tracking changes \n Must conform to the json specification \n Defines all aspects of the spec including blocks, spaces and actions] -->B[MSML Object \n\n JSON file and parsed, with validations and mappings along the way \n Can show different views on the fly]\n    B --> C[Report Outputs \n\n Automatically build reports for the full spec or subviews \n Example: all blocks with an effect on variable XYZ]\n\n```'